In [1]:
import pandas as pd
# from sqlalchemy.ext.automap import automap_base
# from sqlalchemy.orm import Session
# from sqlalchemy import create_engine, func, Table, MetaData
from flask_cors import CORS
import spotipy
from flask import Flask,jsonify
from dotenv import load_dotenv
import os
from flask_cors import CORS
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
load_dotenv()

def get_token():
    cid = os.getenv("CLIENT_ID")
    secret =  os.getenv("CLIENT_SECRET")
    client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    return sp

In [26]:
sp = get_token()
artist_id = sp.search(q='Kanye', type = 'artist')
artist_id

{'artists': {'href': 'https://api.spotify.com/v1/search?query=Kanye&type=artist&offset=0&limit=10',
  'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5K4W6rqBFWDnAN6FQUkS6x'},
    'followers': {'href': None, 'total': 21243558},
    'genres': ['chicago rap', 'hip hop', 'rap'],
    'href': 'https://api.spotify.com/v1/artists/5K4W6rqBFWDnAN6FQUkS6x',
    'id': '5K4W6rqBFWDnAN6FQUkS6x',
    'images': [{'height': 640,
      'url': 'https://i.scdn.co/image/ab6761610000e5eb867008a971fae0f4d913f63a',
      'width': 640},
     {'height': 320,
      'url': 'https://i.scdn.co/image/ab67616100005174867008a971fae0f4d913f63a',
      'width': 320},
     {'height': 160,
      'url': 'https://i.scdn.co/image/ab6761610000f178867008a971fae0f4d913f63a',
      'width': 160}],
    'name': 'Kanye West',
    'popularity': 89,
    'type': 'artist',
    'uri': 'spotify:artist:5K4W6rqBFWDnAN6FQUkS6x'},
   {'external_urls': {'spotify': 'https://open.spotify.com/artist/69UypehHabb68utzfjAV

In [29]:
song_data = sp.search(q='homecoming', type = 'track', limit = 1)
song_id = song_data['tracks']['items'][0]['id']

In [31]:
track = sp.track(song_id)
track

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5K4W6rqBFWDnAN6FQUkS6x'},
    'href': 'https://api.spotify.com/v1/artists/5K4W6rqBFWDnAN6FQUkS6x',
    'id': '5K4W6rqBFWDnAN6FQUkS6x',
    'name': 'Kanye West',
    'type': 'artist',
    'uri': 'spotify:artist:5K4W6rqBFWDnAN6FQUkS6x'}],
  'available_markets': ['AR',
   'AU',
   'AT',
   'BE',
   'BO',
   'BR',
   'BG',
   'CA',
   'CL',
   'CO',
   'CR',
   'CY',
   'CZ',
   'DK',
   'DO',
   'DE',
   'EC',
   'EE',
   'SV',
   'FI',
   'FR',
   'GR',
   'GT',
   'HN',
   'HK',
   'HU',
   'IS',
   'IE',
   'IT',
   'LV',
   'LT',
   'LU',
   'MY',
   'MT',
   'MX',
   'NL',
   'NZ',
   'NI',
   'NO',
   'PA',
   'PY',
   'PE',
   'PH',
   'PL',
   'PT',
   'SG',
   'SK',
   'ES',
   'SE',
   'CH',
   'TW',
   'TR',
   'UY',
   'US',
   'GB',
   'AD',
   'LI',
   'MC',
   'ID',
   'JP',
   'TH',
   'VN',
   'RO',
   'IL',
   'ZA',
   'SA',
   'AE',
   'BH',
   'QA',
   'OM',
 

In [33]:
track['album']['name']

'Graduation'

In [34]:
track['album']['images'][1]['url']

{'height': 300,
 'url': 'https://i.scdn.co/image/ab67616d00001e0226f7f19c7f0381e56156c94a',
 'width': 300}

In [6]:
artist_id['artists']['items'][0]['followers']['total']

21243558

In [7]:
artist_id['artists']['items'][0]['images'][0]['url']

'https://i.scdn.co/image/ab6761610000e5eb867008a971fae0f4d913f63a'

In [8]:
artist_id['artists']['items'][0]['name']

'Kanye West'

In [11]:
artist_id = artist_id['artists']['items'][0]['uri']

In [ ]:

    # create a Spotipy instance with the access token
    sp = get_token()

    artist_id = sp.search(q=artist, type = 'artist')
    artist_id = artist_id['artists']['items'][0]['uri']

    artist_dict = {
        'artist': artist_id['artists']['items'][0]['name'],
        'artist_image': artist_id['artists']['items'][0]['images'][0]['url'],
        'artist_followers': artist_id['artists']['items'][0]['followers']['total']
    }
    
    top_tracks = sp.artist_top_tracks(artist_id, country = 'US')

    song_list = []
    song_id = []

    # Loop through the list to save recesary information about top songs
    for song in top_tracks['tracks']:
    # print(song['name'])
    # print(song['id'])
        song_id.append(song['id'])
        for artist in song['artists']:
        # print(artist['name'])
        # print("------------")
            dict = {
                'name' : song['name'],
                'artist' : artist['name'],
                'id': song['id'],
                'popularity' : song['popularity']
            }
            song_list.append(dict)


    # Get audiofeatures from top songs
    top_artist_features = sp.audio_features(song_id)

    # Create DataFrames
    df_top_artist_audiofeatures = pd.DataFrame(top_artist_features)



    # Create additional dataFrames and drop duplicates
    df_top_artist_song_list = pd.DataFrame(song_list)
    df_top_artist_song_list = df_top_artist_song_list.drop_duplicates(subset=['name']).reset_index(drop=True)



    # Merge DF and add seconds column
    top_artist_df = pd.merge(df_top_artist_song_list, df_top_artist_audiofeatures, on ='id')
    top_artist_df['duration_s'] = top_artist_df['duration_ms'] / 1000
    top_artist_df = top_artist_df[[
        'name','artist','popularity','danceability','energy','loudness','speechiness','acousticness',
        'instrumentalness','liveness','valence','tempo','duration_s']]

    top_artist_songs = top_artist_df.values.tolist() 

    top_song_list = []


    for song in top_artist_songs:
        dict = {
            "song": song[0],
            "artist": song[1],
            "popularity": song[2],
            "danceability": song[3],
            "energy": song[4],
            "loudness": song[5],
            "speechiness": song[6],
            "acousticness": song[7],
            "instrumentalness": song[8],
            "liveness": song[9],
            "valence":song[10],
            "tempo": song[11],
            "duration": song[12],
        }
        top_song_list.append(dict)

    data = top_song_list
    return jsonify(data)

In [12]:
top_tracks = sp.artist_top_tracks(artist_id, country = 'US')
top_tracks

{'tracks': [{'album': {'album_type': 'album',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5K4W6rqBFWDnAN6FQUkS6x'},
      'href': 'https://api.spotify.com/v1/artists/5K4W6rqBFWDnAN6FQUkS6x',
      'id': '5K4W6rqBFWDnAN6FQUkS6x',
      'name': 'Kanye West',
      'type': 'artist',
      'uri': 'spotify:artist:5K4W6rqBFWDnAN6FQUkS6x'}],
    'external_urls': {'spotify': 'https://open.spotify.com/album/4SZko61aMnmgvNhfhgTuD3'},
    'href': 'https://api.spotify.com/v1/albums/4SZko61aMnmgvNhfhgTuD3',
    'id': '4SZko61aMnmgvNhfhgTuD3',
    'images': [{'height': 640,
      'url': 'https://i.scdn.co/image/ab67616d0000b27326f7f19c7f0381e56156c94a',
      'width': 640},
     {'height': 300,
      'url': 'https://i.scdn.co/image/ab67616d00001e0226f7f19c7f0381e56156c94a',
      'width': 300},
     {'height': 64,
      'url': 'https://i.scdn.co/image/ab67616d0000485126f7f19c7f0381e56156c94a',
      'width': 64}],
    'is_playable': True,
    'name': 'Graduation',


In [20]:
top_tracks['tracks'][0]['album']['images'][2]['url']

'https://i.scdn.co/image/ab67616d0000485126f7f19c7f0381e56156c94a'

In [24]:
top_tracks['tracks'][0]['id']

'5TRPicyLGbAF2LGBFbHGvO'

In [25]:

recommended_song_id = []
recommended_song_list = []
for song in top_tracks:
    recommended_song_id.append(song['id'])
    for artist in song['artists']:
    # print(artist['name'])
    # print("------------")
        dict = {
            'name' : song['name'],
            'artist' : artist['name'],
            'id': song['id'],
            'popularity' : song['popularity'],

        }
        recommended_song_list.append(dict)
return {
    'song_id': recommended_song_id,
    'song_list': recommended_song_list
    }

TypeError: string indices must be integers